<a href="https://colab.research.google.com/github/cesarofuchi/python_projects/blob/master/TabelaSanepar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)


```
# Isto está formatado como código
```



# Geração de tabela com rodizio de água por bairro em Curitiba


A seca pelo qual Curitiba e região metropolitana esta passando pode estar longe de acabar. O rodiízio intermitente de água sempre é uma questão no condomínio. 
 
![picture](https://github.com/cesarofuchi/python_projects/blob/main/sanepar/img/banner_20porcento.jpg?raw=true)
*texto em itálico*

Em nossa jornada rumo à automação de tarefas repetitivas em nossa rotina de trabalho, vamos aprender a extrair e utilizar informações úteis de nossos documentos.

Mais uma vez utilizando a biblioteca PDF Plumber vamos extrair e manipular informações de diversos relatórios em pdf ao mesmo tempo.

Para isso, vamos precisar fazer o download dos arquivos de apoio que foram disponibilizados como material complementar dessa aula.

Lembrando: Nativamente, o Google Colab não possui essa biblioteca instalada, então nós precisamos instalar a mesma no nosso ambiente.

Para isso, vamos utilizar o gerenciador de pacotes pip, e com o comando install vamos instalar a biblioteca.

In [ ]:
# instalando o pdfplumber
!pip install pdfplumber

     |████████████████████████████████| 5.6MB 3.2MB/s 
     |████████████████████████████████| 143kB 44.7MB/s 
     |████████████████████████████████| 13.7MB 287kB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.23-cp36-none-any.whl size=29176 sha256=453a73603b7e04f4f15656cb697884ad509a5a3eaacf84aaf8121e63d57a2109
  Stored in directory: /root/.cache/pip/wheels/f6/4d/ae/abb7b92fbed5d09a835c259f26d2a3223ea929a0ad6d322097
Successfully built pdfplumber
